# The Model module

The top module handles the main setup of the instance. 


- Requires:

data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be > 0.4.1
import pyPoseidon
pyPoseidon.__version__

In [ ]:
import pyPoseidon.model as pm

### Graphics

In [ ]:
import pyPoseidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyPoseidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyPoseidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env PATH. This is not needed when we are running from python.

In [ ]:
cpath = pyPoseidon.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

## Setup

The minimum information required is the geomerty boundary. In the most simple case that is a lat/lon box that defines the area of interest. Without loss of generality we select below Iceland as a test case. Feel free to modify the coordinates. 

In addition, the coastlines need to be provided as internal boundaries. if not the default option is using natural earth intermediate resolution (https://www.naturalearthdata.com/downloads/).

The coastlines can be given as a shapefile or GeoDataFrame.

In [ ]:
#define in a dictionary the properties of the model..
model={'solver':'schism',
     'geometry':{'lon_min' : -25,'lon_max' : -12.,'lat_min' : 56.,'lat_max' : 74.}, 
#     'manning':.12, #uniform manning value
#     'windrot':0.00001, #uniform windrot value
#     'tag':'schism', # optional tag 
#     'grid_generator' : 'gmsh', # set grid generator  
#     'bgmesh':'auto', # option for background mesh size
     'start_date':'2017-10-1 0:0:0',
     'time_frame':'12H',
     'meteo_source' : ['./data/erai.grib'], #path to meteo files
     'meteo_engine':'cfgrib',
     'dem_source' : './data/dem.nc', #path to dem file
#     'ncores': 4 , # optional number of cores
     'rpath':'./test/schism/', #location of calc folder
#     'update':['all'], #set which component should be updated  (meteo,dem,model)
#     'monitor':True, # get time series for observation points
#     'parameters':{'dt':400, 'rnday':.5, 'hotout':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'hotout_write':108 } # set param.in components
    }

## Initialize

In [ ]:
#initialize a model
b = pm(**model)

## Step by Step process

In [ ]:
b.create()

In [ ]:
b.output()

In [ ]:
b.save() # saves the json model file

In [ ]:
b.set_obs() # setup station points

In [ ]:
b.run() # execute

## ... or simply in one step


In [ ]:
#initialize a model
b = pm(**model)
b.execute()

The various datasets incorporated in the model can be accessed indepententy as attributes, namely :

- Meteo

In [ ]:
#plt.figure() # xarray uses existing ax if any. You might need to create a new figure to do it separately.
b.meteo.Dataset.msl[2,:,:].plot()

- Grid

In [ ]:
b.grid.Dataset.hplot.grid(lw=.5, markersize=.2) 

- Dem

In [ ]:
b.dem.Dataset.elevation.plot()

## Output

The output of the simulations could be in separate files (due to MPI) or specific folders. pyPoseidon can incorporate the resulting Datasets with 

In [ ]:
b.get_data() # integrate output

In [ ]:
out = b.data.Dataset

In [ ]:
out

In [ ]:
out.depth.compute().max(), out.depth.compute().min()

In [ ]:
out.pplot.grid(lw=.3, markersize=1)

In [ ]:
out.pplot.contourf(var='depth' ,vmin=out.depth.min(), vmax=out.depth.max())

In [ ]:
out.pplot.quiver(it = 2, u=out.hvel.values[2,:,0,0],v=out.hvel.values[2,:,0,1],scale=.01, title='hvel', color='k')

### with hvplot

In [ ]:
out.hplot.contourf(var='depth' , vmin=out.depth.min(), vmax=out.depth.max())#,tiles=True)

In [ ]:
out.hplot.grid(width=800, height=400,tiles=True)

### Animate

In [ ]:
v = out.pplot.frames(var='elev',title='SSH') # change to hplot for holoviews
v

In [ ]:
v.save('test/elev.mp4')

In [ ]:
out.pplot.qframes(u=out.hvel.values[:,:,0,0],v=out.hvel.values[:,:,0,1],title='Vel',scale=.01)#, color='w')

## Read Model

If a model is created by pyPoseidon then there is a .json file that describes the model. One can read into pyPoseidon such a model with 

In [ ]:
a = pyPoseidon.read_model('./test/schism/schism_model.json')

In [ ]:
#a.execute()

However, there might be a model created by other means. pyPoseidon does it's best to incorporate such model with

In [ ]:
c = pm(solver='schism', rfolder = './test/schism/', load_grid=True, load_meteo=True)

In [ ]:
c.grid.Dataset

In [ ]:
c.meteo.Dataset